In [1]:
import torch
import numpy as np
from scipy.spatial.distance import pdist
from scipy.stats import spearmanr
from tqdm.notebook import tqdm
import random

def pairwise_distances_torch(X: torch.Tensor, distance: str = "euclidean") -> torch.Tensor:
    """
    Compute pairwise distances using PyTorch.
    Supported distances: 'euclidean', 'cosine'
    """
    if distance == "euclidean":
        norm = (X ** 2).sum(1).view(-1, 1)
        dist_squared = norm + norm.t() - 2.0 * X @ X.t()
        dist_squared = torch.clamp(dist_squared, min=0)
        return torch.sqrt(dist_squared + 1e-8)
    elif distance == "cosine":
        X = torch.nn.functional.normalize(X, p=2, dim=1)
        sim = X @ X.t()
        return 1 - sim  # cosine distance = 1 - cosine similarity
    else:
        raise ValueError(f"Unsupported distance metric: {distance}")

def topographic_similarity_gpu(
    embeddings: torch.Tensor,
    targets: torch.Tensor,
    subset_size: int = 10_000,
    seeds: list = [0, 1, 2, 3, 4],
    device: str = "cuda",
    distance = "euclidean"
):
    """
    GPU-accelerated topographic similarity using PyTorch and averaging across seeds.

    Returns:
    - float: average Spearman correlation
    """
    assert embeddings.shape[0] == targets.shape[0]

    #embeddings = embeddings.to(device)
    #targets = targets.to(device)

    sims = []

    for seed in tqdm(seeds):
        torch.manual_seed(seed)
        idx = torch.randperm(embeddings.shape[0], device="cpu")[:subset_size]

        emb_sub = embeddings[idx].cuda()
        tgt_sub = targets[idx].cuda()

        # Compute pairwise distances
        dist_emb = pairwise_distances_torch(emb_sub, distance=distance)
        dist_tgt = pairwise_distances_torch(tgt_sub, distance="euclidean")

        # Convert upper triangle to vector for Spearman
        triu_indices = torch.triu_indices(subset_size, subset_size, offset=1)
        emb_vec = dist_emb[triu_indices[0], triu_indices[1]].cpu().numpy()
        tgt_vec = dist_tgt[triu_indices[0], triu_indices[1]].cpu().numpy()

        sim, _ = spearmanr(emb_vec, tgt_vec)
        sims.append(sim)

    return float(np.mean(sims)), sims

def generate_seeds(n: int, base_seed: int = 42) -> list:
    """
    Generate a list of `n` reproducible integer seeds.

    Parameters:
    - n: Number of seeds to generate
    - base_seed: Base seed to initialize RNG for reproducibility

    Returns:
    - List of ints: deterministic random seeds
    """
    rng = random.Random(base_seed)
    return [rng.randint(0, 2**32 - 1) for _ in range(n)]

In [2]:
data = torch.load("3dshapes/3dshapes.pth")

In [3]:
import pandas as pd
seeds = generate_seeds(10)

models = ['vit_b_16', 'vit_b_32', 'vit_l_16','vit_l_32']
results = pd.DataFrame()
for model in models:
    reps = torch.load(f"3dshapes/3dshapes_images_feats_{model}.pth")
    reps = reps - reps.mean(dim=0)
    reps = torch.nn.functional.normalize(reps, dim=-1, p=2.0, eps=1e-12)
    _, sims = topographic_similarity_gpu(
        reps,
        data['latent_ids'].float(),
        subset_size=1_000,
        seeds = seeds)

    results = pd.concat([results, pd.DataFrame({'model': [model]*len(seeds) ,'seed': seeds, 'top_sim': sims})])

results.to_csv("topsim_baselines.csv")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
results

,model,seed,top_sim
0,vit_b_16,2746317213,0.102451
1,vit_b_16,1181241943,0.113070
2,vit_b_16,958682846,0.116893
3,vit_b_16,3163119785,0.108330
4,vit_b_16,1812140441,0.108334
5,vit_b_16,127978094,0.107799
6,vit_b_16,939042955,0.123753
7,vit_b_16,2340505846,0.103413
8,vit_b_16,946785248,0.107528
9,vit_b_16,2530876844,0.119648


In [5]:
grouped = results.groupby('model')['top_sim'].mean().reset_index()

grouped

,model,top_sim
0,vit_b_16,0.111122
1,vit_b_32,0.094166
2,vit_l_16,0.147442
3,vit_l_32,0.126313


## LSM - Top Sim

In [15]:
import pandas as pd
from models import create_model, LightningVersatile
from utils import get_args
from torch.utils.data import TensorDataset, DataLoader
# load model

def get_reps(args, model, dl):
    reps = []
    with torch.no_grad():
        for rep in tqdm(dl):
            bs = rep[0].shape[0]
            rep = rep[0].cuda()
            delta = torch.zeros(bs, len(args.FOVS_PER_DATASET)).cuda()
            new_rep = model.modulator(rep, delta)
            reps.append(new_rep.cpu())
        return torch.cat(reps, dim=0)
    

exps = ["7l4txq1v", #  rep_train_plus extrapolation
        "xg98v4xi"
        ] 

seeds = generate_seeds(10)
subset_size = 10000
results = pd.DataFrame()
for exp_id in tqdm(exps):
    args = get_args(exp_id)

    # Load model with pretrained weights
    encoder, modulator, regressor = create_model(args)
    model = LightningVersatile.load_from_checkpoint(checkpoint_path=f"results/{args.dataset}/{exp_id}/last.ckpt", 
                                        args=args, 
                                        encoder=encoder, 
                                        modulator=modulator,
                                        regressor=regressor)
    print(args.pretrained_reps)

    # Create Subset Datasets
    reps = torch.load(f"3dshapes/3dshapes_images_feats_{args.pretrained_reps}.pth")
    latents =  data['latent_ids']
    sims = []
    for seed in tqdm(seeds):
        torch.manual_seed(seed)
        idx = torch.randperm(reps.shape[0], device="cpu")[:subset_size]
        subset_reps = reps[idx]
        subset_latents = latents[idx]
        ds = TensorDataset(subset_reps)
        dl = DataLoader(ds, batch_size=1024, shuffle=False)
        new_reps = get_reps(args, model, dl)

        _, sim = topographic_similarity_gpu(
            new_reps,
            subset_latents.float(),
            subset_size=subset_size,
            seeds = [seed],
            distance = "cosine")
        sims.append(sim[0])
    print(sims)
    results = pd.concat([results, pd.DataFrame({'model': [f"lsp_{args.sub_dataset}_{args.pretrained_reps}"]*len(seeds) ,'seed': seeds, 'top_sim': sims})])
results.to_csv("topsim_lsm_baselines.csv")

  0%|          | 0/2 [00:00<?, ?it/s]

vit_l_32


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[-0.007915091592392903, -0.007612920641561018, -0.008739540471947686, -0.009990558310293529, -0.009889535831871557, -0.008901011117443005, -0.009584788824939865, -0.007432201619869495, -0.00852653286840817, -0.004247163764604285]


In [16]:
sims

[-0.007915091592392903,
 -0.007612920641561018,
 -0.008739540471947686,
 -0.009990558310293529,
 -0.009889535831871557,
 -0.008901011117443005,
 -0.009584788824939865,
 -0.007432201619869495,
 -0.00852653286840817,
 -0.004247163764604285]

In [17]:
df1 = pd.read_csv("topsim_baselines.csv")
df2 = pd.read_csv("topsim_lsm_baselines.csv")

results = pd.concat([df1,df2])
grouped = results.groupby('model')['top_sim'].mean().reset_index()
grouped

,model,top_sim
0,lsp_composition_vit_l_32,-0.008284
1,lsp_extrapolation_vit_l_32,0.063193
2,vit_b_16,0.111122
3,vit_b_32,0.094166
4,vit_l_16,0.147442
5,vit_l_32,0.126313


In [18]:
print(grouped.to_latex(index=None).replace("_","\_"))

\begin{tabular}{llr}
\toprule
 & model & top\_sim \\
\midrule
0 & lsp\_composition\_vit\_l\_32 & -0.008284 \\
1 & lsp\_extrapolation\_vit\_l\_32 & 0.063193 \\
2 & vit\_b\_16 & 0.111122 \\
3 & vit\_b\_32 & 0.094166 \\
4 & vit\_l\_16 & 0.147442 \\
5 & vit\_l\_32 & 0.126313 \\
\bottomrule
\end{tabular}



In [37]:
a[0].shape

torch.Size([1000, 768])

In [41]:
new_reps.shape

torch.Size([1000, 128])

In [6]:
grouped

,model,top_sim
0,lsp_LightningRepClassification(\n (modulator)...,0.055381
1,lsp_LightningRepClassification(\n (modulator)...,0.082469


In [2]:
data = torch.load("3dshapes/3dshapes.pth")

In [6]:
import torch
import itertools
import pandas as pd
from collections import defaultdict
import random
from tqdm.notebook import tqdm
from utils import get_args
def fast_parallelism_score(reps, latents, factor_idx, num_pairs=1000, max_group_size=500):
    """
    Computes parallelism score for a given latent factor, scalable to large datasets.
    """

    other_indices = [i for i in range(latents.shape[1]) if i != factor_idx]
    reps = reps.cpu()
    latents = latents.cpu()

    # Group samples by all other factor values
    groups = defaultdict(list)
    for idx in range(len(latents)):
        key = tuple(latents[idx, other_indices].tolist())
        groups[key].append(idx)

    deltas = []
    rng = random.Random(42)

    # For each group (fixed other factors), find pairs that differ in factor_idx
    for group in groups.values():
        if len(group) < 2:
            continue
        group_latents = latents[group, factor_idx]
        group_reps = reps[group]
        val_to_indices = defaultdict(list)

        for i, val in zip(group, group_latents.tolist()):
            val_to_indices[val].append(i)

        # Get all combinations of different values for this factor
        for v1, v2 in itertools.combinations(val_to_indices.keys(), 2):
            idx1 = val_to_indices[v1]
            idx2 = val_to_indices[v2]

            for i1 in idx1[:max_group_size]:
                for i2 in idx2[:max_group_size]:
                    delta = reps[i1] - reps[i2]
                    deltas.append(delta)
                    if len(deltas) >= num_pairs:
                        break
                if len(deltas) >= num_pairs:
                    break
            if len(deltas) >= num_pairs:
                break
        if len(deltas) >= num_pairs:
            break

    if len(deltas) < 2:
        return float("nan")

    deltas = torch.stack(deltas)
    deltas = torch.nn.functional.normalize(deltas, dim=1)
    deltas = deltas.to("cuda")
    sims = torch.matmul(deltas, deltas.T).cpu()
    upper = sims.triu(diagonal=1)
    score = upper.sum() / (upper != 0).sum()
    return score.item()

models = ['vit_b_16', 'vit_b_32', 'vit_l_16','vit_l_32']
factors = ['floor_hue','wall_hue','object_hue','scale','shape', 'orientation']
results = []
latents = data['latent_ids']
for model in tqdm(models):
    reps = torch.load(f"3dshapes/3dshapes_images_feats_{model}.pth")
    for factor_idx in tqdm(range(6)):
        score = fast_parallelism_score(reps, latents, factor_idx, num_pairs=5000, max_group_size=2500)
        results.append({
            "model": model,
            "factor": factors[factor_idx],
            "parallelism_score": score
        })
    p_score = fast_parallelism_score(reps, latents, 0)

# Convert to DataFrame
df_scores = pd.DataFrame(results)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
df_scores.to_csv("pscores_baselines.csv")

In [8]:
df_scores

,model,factor,parallelism_score
0,vit_b_16,floor_hue,0.107740
1,vit_b_16,wall_hue,0.106162
2,vit_b_16,object_hue,0.089541
3,vit_b_16,scale,0.122251
4,vit_b_16,shape,0.218613
5,vit_b_16,orientation,0.054536
6,vit_b_32,floor_hue,0.100278
7,vit_b_32,wall_hue,0.113571
8,vit_b_32,object_hue,0.099302
9,vit_b_32,scale,0.145598


In [9]:
from models import create_model, LightningVersatile
from torch.utils.data import TensorDataset, DataLoader
# def get_rep(input_reps, model, idxs):
#     reps = []
#     subset_reps = input_reps[idxs]
#     ds = TensorDataset(subset_reps)
#     dl = DataLoader(ds, batch_size=1024, shuffle=False)
    
#     with torch.no_grad():
#         for rep in tqdm(dl):
#             bs = rep[0].shape[0]
#             rep = rep[0].cuda()
#             delta = torch.zeros(bs, 6).cuda()
#             new_rep = model.modulator(rep, delta)
#             reps.append(new_rep.cpu())
#         return torch.cat(reps, dim=0)

def get_rep(input_reps, model, idxs):
    reps = []
    subset_reps = input_reps[idxs].cuda()
    #dl = DataLoader(ds, batch_size=1024, shuffle=False)
    
    with torch.no_grad():
        delta = torch.zeros(1,6).cuda()
        new_rep = model.modulator(subset_reps, delta)
        return new_rep
        
def fast_parallelism_score_lazy(model, get_rep, reps, latents, factor_idx, num_pairs=1000, max_group_size=500):

    other_indices = [i for i in range(latents.shape[1]) if i != factor_idx]
    latents = latents.cpu()

    groups = defaultdict(list)
    for idx in range(len(latents)):
        key = tuple(latents[idx, other_indices].tolist())
        groups[key].append(idx)

    deltas = []
    rng = random.Random(42)

    for group in groups.values():
        if len(group) < 2:
            continue
        group_latents = latents[group, factor_idx]
        val_to_indices = defaultdict(list)
        for i in group:
            val = latents[i, factor_idx].item()
            val_to_indices[val].append(i)

        for v1, v2 in itertools.combinations(val_to_indices.keys(), 2):
            idx1 = val_to_indices[v1][:max_group_size]
            idx2 = val_to_indices[v2][:max_group_size]
            reps1 = get_rep(reps, model, idx1)
            reps2 = get_rep(reps, model, idx2)

            for i1, r1 in zip(idx1, reps1):
                for i2, r2 in zip(idx2, reps2):
                    deltas.append(r1 - r2)
                    if len(deltas) >= num_pairs:
                        break
                if len(deltas) >= num_pairs:
                    break
            if len(deltas) >= num_pairs:
                break
        if len(deltas) >= num_pairs:
            break

    if len(deltas) < 2:
        return float("nan")

    deltas = torch.stack(deltas)
    deltas = torch.nn.functional.normalize(deltas, dim=1)
    sims = torch.matmul(deltas, deltas.T)
    upper = sims.triu(diagonal=1)
    score = upper.sum() / (upper != 0).sum()
    return score.item()

models = ['vit_b_16', 'vit_b_32', 'vit_l_16','vit_l_32']
factors = ['floor_hue','wall_hue','object_hue','scale','shape', 'orientation']
results = []
latents = data['latent_ids']
exps = ["7l4txq1v", #  rep_train_plus extrapolation
        "xg98v4xi"
        ] 
for exp_id in tqdm(exps):
    args = get_args(exp_id)

    # Load model with pretrained weights
    encoder, modulator, regressor = create_model(args)
    model = LightningVersatile.load_from_checkpoint(checkpoint_path=f"results/{args.dataset}/{exp_id}/last.ckpt", 
                                        args=args, 
                                        encoder=encoder, 
                                        modulator=modulator,
                                        regressor=regressor)
    print(args.pretrained_reps)

    # Create Subset Datasets
    reps = torch.load(f"3dshapes/3dshapes_images_feats_{args.pretrained_reps}.pth")
    reps = reps - reps.mean(dim=0)
    reps = torch.nn.functional.normalize(reps, dim=-1, p=2.0, eps=1e-12)
    
    for factor_idx in tqdm(range(6)):
        score = fast_parallelism_score_lazy(model, get_rep, reps, latents, factor_idx, num_pairs=5000, max_group_size=2500)
        results.append({
            "model": f"lsm_{args.sub_dataset}_{args.pretrained_reps}",
            "factor": factors[factor_idx],
            "parallelism_score": score
        })
    p_score = fast_parallelism_score(reps, latents, 0)

# Convert to DataFrame
df_scores = pd.DataFrame(results)

/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
There was a problem when trying to write in your cache folder (/storage/cache). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


  0%|          | 0/2 [00:00<?, ?it/s]

vit_l_32


  0%|          | 0/6 [00:00<?, ?it/s]

vit_l_32


  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
df_scores.to_csv("pscores_lsm_baselines.csv")

In [11]:
df_scores

,model,factor,parallelism_score
0,lsm_extrapolation_vit_l_32,floor_hue,0.002826
1,lsm_extrapolation_vit_l_32,wall_hue,0.002256
2,lsm_extrapolation_vit_l_32,object_hue,0.004797
3,lsm_extrapolation_vit_l_32,scale,0.122548
4,lsm_extrapolation_vit_l_32,shape,0.049072
5,lsm_extrapolation_vit_l_32,orientation,0.025158
6,lsm_composition_vit_l_32,floor_hue,0.114055
7,lsm_composition_vit_l_32,wall_hue,0.079252
8,lsm_composition_vit_l_32,object_hue,0.274705
9,lsm_composition_vit_l_32,scale,0.238835


In [12]:
df1 = pd.read_csv("pscores_baselines.csv")
df2 = pd.read_csv("pscores_lsm_baselines.csv")

results = pd.concat([df1,df2])

In [13]:
results = results.pivot_table(
        index=['model'],
        columns='factor',
        values='parallelism_score'
    ).reset_index()

In [53]:
def df_to_tabularx_parallelism(df, label, caption, column_width='\\textwidth'):
    import io
    import pandas as pd

    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)

    df_fmt = df.copy()

    # Drop 'factor' column if present
    if 'factor' in df_fmt.columns:
        df_fmt = df_fmt.drop(columns=['factor'])

    # Escape underscores and capitalize model names
    df_fmt['model'] = df_fmt['model'].astype(str).map(lambda s: s.replace('_', r'\_').upper())

    # Format float columns to 2 decimal places
    for col in df_fmt.columns:
        if col != 'model':
            df_fmt[col] = pd.to_numeric(df_fmt[col], errors='coerce').map(lambda x: f"{x:.2f}")

    # Column format for tabularx
    column_format = 'l' + 'X' * (df_fmt.shape[1] - 1)
    bold_headers = [f"\\textbf{{{col.replace('_', ' ').capitalize()}}}" for col in df_fmt.columns]

    # Generate LaTeX table without header
    buf = io.StringIO()
    df_fmt.to_latex(
        buf,
        index=False,
        header=False,
        escape=False,  # allow LaTeX commands like \_
        column_format=column_format
    )
    lines = buf.getvalue().splitlines()

    header_line = ' & '.join(bold_headers) + ' \\\\'
    table_body = '\n'.join(lines[3:-2])

    latex = (
        f"\\begin{{table}}[ht]\n"
        f"\\centering\n"
        f"\\caption{{{caption}}}\n"
        f"\\label{{{label}}}\n"
        f"\\begin{{tabularx}}{{{column_width}}}{{{column_format}}}\n"
        f"{header_line}\n"
        f"{table_body}\n"
        f"\\end{{tabularx}}\n"
        f"\\end{{table}}"
    )

    return latex



label = "tab:asda"
caption = "Hola"
print(df_to_tabularx_parallelism(results, label, caption, column_width='\\textwidth'))

\begin{table}[ht]
\centering
\caption{Hola}
\label{tab:asda}
\begin{tabularx}{\textwidth}{lXXXXXX}
\textbf{Model} & \textbf{Floor hue} & \textbf{Object hue} & \textbf{Orientation} & \textbf{Scale} & \textbf{Shape} & \textbf{Wall hue} \\
LSM\_COMPOSITION\_VIT\_B\_32 & 0.07 & 0.02 & 0.01 & 0.07 & 0.10 & 0.06 \\
LSM\_COMPOSITION\_VIT\_L\_32 & 0.16 & 0.04 & 0.10 & 0.05 & 0.17 & 0.05 \\
LSM\_EXTRAPOLATION\_VIT\_B\_32 & 0.02 & 0.09 & 0.05 & 0.28 & 0.25 & 0.08 \\
LSM\_EXTRAPOLATION\_VIT\_L\_32 & 0.02 & 0.03 & 0.01 & 0.12 & 0.13 & 0.02 \\
LSM\_INTERPOLATION\_VIT\_B\_32 & 0.01 & 0.04 & 0.05 & 0.04 & 0.12 & 0.05 \\
LSM\_INTERPOLATION\_VIT\_L\_32 & 0.05 & 0.03 & 0.02 & 0.03 & 0.24 & 0.01 \\
VIT\_B\_16 & 0.11 & 0.09 & 0.05 & 0.12 & 0.22 & 0.11 \\
VIT\_B\_32 & 0.10 & 0.10 & 0.06 & 0.15 & 0.22 & 0.11 \\
VIT\_L\_16 & 0.13 & 0.12 & 0.07 & 0.13 & 0.23 & 0.13 \\
VIT\_L\_32 & 0.11 & 0.11 & 0.05 & 0.13 & 0.18 & 0.11 \\
\end{tabularx}
\end{table}


In [14]:
results

factor,model,floor_hue,object_hue,orientation,scale,shape,wall_hue
0,lsm_composition_vit_l_32,0.114055,0.274705,0.240517,0.238835,0.070536,0.079252
1,lsm_extrapolation_vit_l_32,0.002826,0.004797,0.025158,0.122548,0.049072,0.002256
2,vit_b_16,0.107740,0.089541,0.054536,0.122251,0.218613,0.106162
3,vit_b_32,0.100278,0.099302,0.059499,0.145598,0.218952,0.113571
4,vit_l_16,0.130729,0.124927,0.066765,0.125357,0.233449,0.125484
5,vit_l_32,0.107406,0.108488,0.051214,0.127816,0.178520,0.114928
